# Index   

### Importing libraries

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import  applications
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Conv1D, MaxPool2D,GlobalAveragePooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings('ignore')
import random
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
import glob
import shutil
import os
import itertools
from sklearn.metrics import confusion_matrix ,classification_report

In [3]:
# import pandas as pd 
import os
import pathlib
#import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential



In [ ]:

import tensorflow_hub as hub
import warnings
warnings.filterwarnings('ignore')
os.environ["KMP_WARNINGS"] = "FALSE" 



# Building the neuronal network

In [ ]:
https://medium.com/@himancodes/building-an-image-classifier-from-scratch-using-convolutional-neural-networks-c9ebcde3a53e

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

base = MobileNetV2(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))
tf.keras.backend.clear_session()
    
for layer in base.layers:
    layer.trainable =  False

mobilenet_model = Sequential()
mobilenet_model.add(base)
mobilenet_model.add(GlobalAveragePooling2D())

mobilenet_model.add(Dense(256, activation='relu'))
mobilenet_model.add(Dropout(0.1))

mobilenet_model.add(Dense(256, activation='relu'))
mobilenet_model.add(Dropout(0.1))
mobilenet_model.add(Dense(3, activation='softmax')) #last layer should have as many neuron as options = 3 sports

To implement f1 macro score in Keras should be done in one of the following solutions:

In [ ]:
import tensorflow as tf
import keras.backend as K

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

# https://www.kaggle.com/code/guglielmocamporese/macro-f1-score-keras

In [ ]:
class MetricsCallback(keras.callbacks.Callback):
    def __init__(self):
        super(MetricsCallback, self).__init__()
    def  on_train_begin(self,logs={}):
      self.f1_macro=[]
    def on_epoch_end(self, epoch, logs=None):
      y_pred=self.model.predict(x_test).round()
      y_true=y_test
      score=f1_score(y_true, y_pred, average='macro')
      self.f1_macro.append(score)
      print(" F1 macro :",score)

metrics=MetricsCallback()

model.fit(x_train, y_train,validation_data=(x_test,y_test),batch_size=batch_size, epochs=5,callbacks=[metrics])

# https://colab.research.google.com/drive/1hTf0LjO-h2l6diVkvj0gQme25clUdFt2#scrollTo=S3hxXkGocmUq
# https://androidkt.com/calculate-f1-macro-in-keras/

# multiclass f1 macro score https://towardsdatascience.com/f-beta-score-in-keras-part-ii-15f91f07c9a4


In [ ]:
mobilenet_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy',metrics=['accuracy'])